In [ ]:
'''
This is a script for testing if the tensorflow nodes works. Run the following commands to set up the system:
roslaunch fetch_coppeliasim simulation.launch
roslaunch fetch_coppeliasim fetch_control.launch
rosrun manipulation_test grasp_visualizer
[with the correct conda environment] rosrun ros_tensorflow grasp_prediction_server.py 
[with the correct conda environment] rosrun ros_tensorflow CoM_prediction_server.py 
'''

In [1]:
import rospy
import tf2_ros
from ros_tensorflow_msgs.srv import *
from rail_segmentation.srv import *
from rail_manipulation_msgs.srv import *
from geometry_msgs.msg import TransformStamped
from manipulation_test.srv import *
import numpy as np

In [2]:
rospy.init_node('example_node')

In [3]:
# find the table first
rospy.wait_for_service('table_searcher/search_table')
table_searcher = rospy.ServiceProxy('table_searcher/search_table', SearchTable)

In [4]:
table_info = table_searcher()

In [5]:
# detect obstacle on the table
rospy.wait_for_service('table_searcher/segment_objects')
object_searcher = rospy.ServiceProxy('table_searcher/segment_objects', SegmentObjects)

In [6]:
detected_objects = object_searcher()

In [7]:
print("detected objects' number: ", len(detected_objects.segmented_objects.objects))

detected objects' number:  1


In [8]:
# get camera transform from tf tree
tfBuffer = tf2_ros.Buffer()
listener = tf2_ros.TransformListener(tfBuffer)

In [9]:
try:
    camera_trans = tfBuffer.lookup_transform('base_link', 'head_camera_rgb_optical_frame', rospy.Time())
except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException):
    print("tf error")

In [10]:
rospy.wait_for_service('grasp_predict')

In [18]:
grasp_predictor = rospy.ServiceProxy('grasp_predict', Predict)
try:
    predicted_grasp_result = grasp_predictor(table_info.full_point_cloud, detected_objects.segmented_objects.objects[0].point_cloud, camera_trans)
except rospy.ServiceException as e:
    print("Service call failed: %s"%e)

In [19]:
# prepare the client to visualize the predicted grasp poses
rospy.wait_for_service('visualize_regrasp')

In [20]:
import random
import numpy as np
from geometry_msgs.msg import PoseStamped

def pose_to_vector(pose_stamped):
    pos = pose_stamped.pose.position
    quat = pose_stamped.pose.orientation

    return np.array([pos.x, pos.y, pos.z, quat.x, quat.y, quat.z, quat.w])

def euclidean_distance(v1, v2):
    return np.linalg.norm(v1 - v2)

def k_means_clustering(poses, k, max_iterations=100):
    # Convert poses to vectors
    pose_vectors = [pose_to_vector(pose) for pose in poses]

    # Randomly initialize centroids
    centroids = random.sample(pose_vectors, k)

    for _ in range(max_iterations):
        # Assign pose_vectors to nearest centroids
        clusters_indices = [[] for _ in range(k)]
        for i, pose_vector in enumerate(pose_vectors):
            closest_centroid_index = np.argmin([euclidean_distance(pose_vector, centroid) for centroid in centroids])
            clusters_indices[closest_centroid_index].append(i)

        # Update centroids
        new_centroids = [np.mean([pose_vectors[i] for i in cluster], axis=0) for cluster in clusters_indices]

        # Check for convergence
        if np.allclose(centroids, new_centroids):
            break

        centroids = new_centroids

    # Create clusters of PoseStamped objects
    clusters = [[poses[i] for i in cluster] for cluster in clusters_indices]

    return clusters[0]

In [21]:
grasp_visualizer = rospy.ServiceProxy('visualize_regrasp', VisualizeRegrasp)
try:
    clustered_grasp_poses = k_means_clustering(predicted_grasp_result.predicted_grasp_poses, 10)
    grasp_visualizer(clustered_grasp_poses, [0.08 for _ in range(10)], [0 for _ in range(10)])
except rospy.ServiceException as e:
    print("Service call failed: %s"%e)

In [22]:
# test center of mass prediction
import sensor_msgs.point_cloud2 as pc2
import sensor_msgs.msg as sensor_msgs
rospy.wait_for_service('CoMPredict')

In [23]:
# generate point cloud for table
# deprecated

# plane_length = detected_objects.segmented_objects.objects[0].bounding_volume.dimensions.y*1.7
# plane_width = detected_objects.segmented_objects.objects[0].bounding_volume.dimensions.z*1.7
# plane_height = detected_objects.segmented_objects.objects[0].bounding_volume.dimensions.x*1.7
# planeResolution = 0.005

plane_pose = detected_objects.segmented_objects.objects[0].bounding_volume.pose

# plane_pc = np.zeros((int(plane_length/planeResolution), int(plane_width/planeResolution), int(plane_height/planeResolution), 3))
# for i in range(int(plane_length/planeResolution)):
#     for j in range(int(plane_width/planeResolution)):
#             plane_pc[i, j, :, :] = [- plane_height / 2.0, i*planeResolution- plane_width / 2.0, j*planeResolution- plane_length / 2.0]

# # transform the point cloud by plane pose
# # convert the quaternion to rotation matrix
# def quat_to_rot(q):
#     w, x, y, z = q
#     return np.array([[1-2*y**2-2*z**2, 2*x*y-2*z*w, 2*x*z+2*y*w],
#                      [2*x*y+2*z*w, 1-2*x**2-2*z**2, 2*y*z-2*x*w],
#                      [2*x*z-2*y*w, 2*y*z+2*x*w, 1-2*x**2-2*y**2]])
# R = quat_to_rot([plane_pose.pose.orientation.w, plane_pose.pose.orientation.x, plane_pose.pose.orientation.y, plane_pose.pose.orientation.z])

# plane_pc = np.matmul(plane_pc, R)
# plane_pc = plane_pc + np.array([plane_pose.pose.position.x, plane_pose.pose.position.y, plane_pose.pose.position.z])

# # encode the point cloud into a ros pc2 message


# plane_pc = plane_pc.reshape(-1, 3)
# plane_pc_msg = pc2.create_cloud_xyz32(plane_pose.header, plane_pc)


In [24]:
data = np.load("data_1_cam_0.npz")
pc = data['pc']
labels = data['labels']
table_pc = pc[labels == 1]
object_pc = pc[labels == 0]

# convert the point cloud to ros pc2 message
table_pc_msg = pc2.create_cloud_xyz32(plane_pose.header, table_pc)
object_pc_msg = pc2.create_cloud_xyz32(plane_pose.header, object_pc)

# create a tf transform for the camera, which the identity matrix
CoM_camera_trans = geometry_msgs.msg.TransformStamped()
CoM_camera_trans.header.stamp = rospy.Time.now()
CoM_camera_trans.header.frame_id = 'camera_link'
CoM_camera_trans.child_frame_id = 'camera_link'
CoM_camera_trans.transform.translation.x = 0.0
CoM_camera_trans.transform.translation.y = 0.0
CoM_camera_trans.transform.translation.z = 0.0
CoM_camera_trans.transform.rotation.x = 0.0
CoM_camera_trans.transform.rotation.y = 0.0
CoM_camera_trans.transform.rotation.z = 0.0
CoM_camera_trans.transform.rotation.w = 1.0

In [27]:
CoM_predictor = rospy.ServiceProxy('CoMPredict', ComPredict)
try:
    CoM_grasp_result = CoM_predictor(table_pc_msg, object_pc_msg, CoM_camera_trans)
except rospy.ServiceException as e:
    print("Service call failed: %s"%e)

In [28]:
print(CoM_grasp_result)

com: 
  x: 0.11123594721547911
  y: -0.04492435990798124
  z: 0.9625998206798431
